In [1]:
import pandas as pd
import random
import sqlalchemy
import scipy.stats as stats
import data
import importlib
import numpy as np
import time

In [2]:
importlib.reload(data)

<module 'data' from 'C:\\Users\\shaur\\Python\\Scouting\\monte-carlo-simulator\\data.py'>

In [3]:
engine = sqlalchemy.create_engine('sqlite:///blockparty22.sqlite3')
measures = pd.read_sql('SELECT * FROM Measures;', engine)
matches = pd.read_sql('SELECT * FROM Matches;', engine)

In [4]:
last_match_for_averages = 'qm41'
matches_for_averages = data.filter_matches(matches, last_match=last_match_for_averages)
measures_for_averages = data.filter_measures(measures, last_match=last_match_for_averages)
matches_to_simulate = data.filter_matches(matches, first_match = 'qm42')

In [5]:
counts = data.get_counts(measures_for_averages)
taxi = data.get_taxi_sums(measures_for_averages)
hangar_totals = data.get_hangar_totals(measures_for_averages)

matches_to_simulate_data = data.get_matches(data.get_averages(taxi, counts, hangar_totals, matches_for_averages), matches_to_simulate)
#move hangar end level 0 column to the end for multinomial distribution purposes (see numpy documentation)
matches_to_simulate_data = matches_to_simulate_data.iloc[:, [i for i in range(9)]+[i for i in range(10,14)]+[9]]

In [6]:
import tba
rankings = tba.get_rankings('2022wamc')
r = [{'team_number': team['team_key'], 'ranking_points': team['extra_stats'][0]} for team in rankings['rankings']]
current_rps = pd.DataFrame(r).set_index('team_number').sort_values(by='team_number')

In [7]:
final_rankings_dict={}
teams = pd.read_sql('SELECT * FROM Teams;', engine)
for team_num in teams.team_number:
    final_rankings_dict[team_num] = 0
final_rankings_dict['frc360'] = 0
final_rankings_dict['frc9992'] = 0
final_rankings_dict['frc9991'] = 0

In [8]:
start = time.time()
for i in range(20):
    scores=matches_to_simulate_data.copy() #0.0633995532989502 seconds per thousand iterations
    scores.taxi = scores.taxi.apply(data.taxi)#1 second per thousand iterations # 1.19270038605 per thousand iterations
    for column in ['auto_upper', 'auto_lower', 'tele_upper', 'tele_lower', 'endgame_penalty_count']: #2.14568018913 per thousand iterations
        scores[column]=stats.poisson.rvs(mu=scores[column])
    for num in range(len(scores)):
        scores.iloc[num, -5:] = np.random.multinomial(1, scores.iloc[num, -5:])
    cargo = (scores.loc[:, ['match','alliance','auto_upper', 'auto_lower', 'tele_upper', 'tele_lower']]
                   .groupby(by=['match', 'alliance'])
                   .sum()
                   .assign(auto=lambda df: df.auto_lower+df.auto_upper,
                           tele = lambda df: df.tele_upper + df.tele_lower)
                   .drop(['auto_lower', 'auto_upper', 'tele_upper', 'tele_lower'], axis = 1)
                   .assign(total = lambda df: df.auto+df.tele)
                   .assign(cargo_rp=lambda df: (df.total>=20)|((df.auto>=5)&(df.total>=18))))


    scores_matrix = scores.iloc[:, 3:].to_numpy()
    values_matrix = np.matrix('2; 4; -4; 1; 2; 2; 4; 6; 10; 15; 0')
    points_matrix = np.dot(scores_matrix, values_matrix)

    hangar_scores_matrix = scores.iloc[:, -5:].to_numpy()
    hangar_values_matrix = np.matrix('4; 6; 10; 15; 0')
    hangar_points_matrix = np.dot(hangar_scores_matrix, hangar_values_matrix)

    scores = scores.assign(points = points_matrix, hangar_points = hangar_points_matrix)


    rps = (scores.groupby(by=['match', 'alliance']).sum()
                .iloc[:, -2:]
                .assign(hangar_rp = lambda df: df.hangar_points>=16)
                .join(cargo))

    def winning_alliance(point_differential):
        if point_differential>0:
            return 'blue'
        elif point_differential<0:
            return 'red'
        else:
            return 'tie'

    winner_series = (pd.DataFrame(scores.groupby(by=['alliance','match']).sum().loc[:, 'points'])
                    .reset_index(level=0)
                    .pivot(columns='alliance')
                    .assign(winner = lambda df:df.points.blue-df.points.red)
                    .winner.apply(winning_alliance))

stop = time.time()
print(stop-start)

0.5515854358673096


In [9]:
for team in final_rankings_dict.keys():
    final_rankings_dict[team] = round(final_rankings_dict[team]/100, 2)
print(dict(sorted(final_rankings_dict.items(), key = lambda item: item[1])))